In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
size = 100
low = 1
hi = 5

In [ ]:
range_ends = np.r_[low, hi]

In [ ]:
θ_0 = np.random.uniform(0, 1)
θ_1 = np.random.uniform(0, 3)

solution = np.vectorize(lambda x: θ_0 + θ_1 * x)

In [ ]:
θ_0, θ_1

In [ ]:
xs = np.random.uniform(low, hi, size)
y_noise = np.random.randn(size)
ys = θ_0 + θ_1 * xs + y_noise 

In [ ]:
plt.plot(xs, ys, '.r')

In [ ]:
opt_θ_1 = (ys.mean() - (xs * ys).sum() / xs.sum()) / (xs.mean() - (xs ** 2).sum() / xs.sum())
opt_θ_0 = ys.mean() - opt_θ_1 * xs.mean()

h = np.vectorize(lambda x: opt_θ_0 + opt_θ_1 * x)

In [ ]:
opt_θ_0, opt_θ_1

In [ ]:
print("squared error = ", (θ_0 - opt_θ_0) ** 2 + (θ_1 - opt_θ_1) ** 2)

In [ ]:
plt.plot(xs, ys, '.r')
plt.plot(range_ends, h(range_ends))
plt.plot(range_ends, solution(range_ends))

In [ ]:
J = np.vectorize(lambda θ_0, θ_1: ((θ_0 + θ_1 * xs - ys) ** 2).sum() / xs.size)

In [ ]:
θ_0_range = np.r_[θ_0 - 1 : θ_0 + 1 : 100j]
θ_1_range = np.r_[θ_1 - 1 : θ_1 + 1 : 100j]
plt.contour(θ_1_range, θ_0_range, J(θ_0_range[:,None], θ_1_range[None,:]))
